In [ ]:
!pip install cookiecutter


In [ ]:
!cookiecutter -c v1 https://github.com/drivendata/cookiecutter-data-science


In [ ]:
!ls


In [ ]:
%cd /content/EXAMEN_PARCIAL_AA_CASSANO_NANCY_JULIETA
!ls
!find . -maxdepth 2 -type d | sort


In [ ]:
from google.colab import files
uploaded = files.upload()




In [ ]:
!mkdir -p data/raw
!mv dataset_prestaciones_extrahospitalarias_2025_realista_v1.csv data/raw/
!ls data/raw

In [ ]:
!git init
!git config user.name "julietcass71"
!git config user.email "doctoracassano@gmail.com"

!git add .
!git commit -m "Scaffold Cookiecutter + dataset (Entrega 2)"

!git branch -M main
!git remote add origin https://github.com/julietcass71/EXAMEN_PARCIAL_AA_CASSANO_NANCY_JULIETA.git


In [ ]:
!git pull --rebase origin main || git pull origin main --allow-unrelated-histories


In [ ]:
!git push -u origin main


In [ ]:
%cd /content/EXAMEN_PARCIAL_AA_CASSANO_NANCY_JULIETA
!git rebase --abort || echo "No había rebase activo"


In [ ]:
# Traigo el remoto permitiendo historias no relacionadas (porque el repo remoto tenía archivos previos)
!git pull origin main --allow-unrelated-histories || true

# Resolver conflictos quedándome con mis archivos (los de Cookiecutter)
!git checkout --ours .gitignore
!git checkout --ours LICENSE
!git checkout --ours README.md

# Marcar como resuelto y confirmar el merge
!git add .gitignore LICENSE README.md
!git commit -m "Merge remote: keep Cookiecutter versions of .gitignore, LICENSE, README"


In [ ]:
!git fetch origin
!git merge -X ours origin/main --allow-unrelated-histories -m "Merge origin/main (prefer local cookiecutter files)"


In [ ]:
!git push -u origin main


In [ ]:
!git remote set-url origin https://github.com/julietcass71/EXAMEN_PARCIAL_AA_CASSANO_NANCY_JULIETA.git


In [ ]:
!ls -lh data/raw | sed -n '1,50p'

In [ ]:
!git add -f data/raw/dataset_prestaciones_extrahospitalarias_2025_realista_v1.csv
!git commit -m "Add raw dataset (override Cookiecutter ignore for course submission)"
!git push

In [ ]:
# Autenticación (oculta)
import getpass
token = getpass.getpass('Pegá tu GitHub PAT: ')


In [ ]:
# Empujar commits (incluye el CSV)
!git push -u origin main




In [ ]:
from pathlib import Path

PROJ = Path(".").resolve()

RAW = PROJ / "data" / "raw" / "dataset_prestaciones_extrahospitalarias_2025_realista_v1.csv"
PROC_DIR = PROJ / "data" / "processed"
PROC = PROC_DIR / "dataset_prestaciones_extrahospitalarias_2025_realista_v1.csv"
META = PROC.with_name(PROC.stem + "_metadata.json")

print("Proyecto:", PROJ)
print("RAW existe?:", RAW.exists())

In [ ]:
!git clone https://github.com/julietcass71/EXAMEN_PARCIAL_AA_CASSANO_NANCY_JULIETA.git
%cd EXAMEN_PARCIAL_AA_CASSANO_NANCY_JULIETA
!ls -la
!ls -la data/raw


In [ ]:
# Procesa el dataset raw: processed y genera metadata.json
from pathlib import Path
import pandas as pd, hashlib, json

PROJ = Path(".").resolve()
RAW = PROJ / "data" / "raw" / "dataset_prestaciones_extrahospitalarias_2025_realista_v1.csv"
PROC_DIR = PROJ / "data" / "processed"
PROC = PROC_DIR / RAW.name
META = PROC.with_name(PROC.stem + "_metadata.json")

print("Archivo RAW encontrado:", RAW.exists())
print("Procesando...")

# Crear carpeta processed
PROC_DIR.mkdir(parents=True, exist_ok=True)

# Leer CSV
df = pd.read_csv(RAW)

# Guardar copia procesada (solo verificaR integridad)
df.to_csv(PROC, index=False, encoding="utf-8")

# Calcular MD5
def md5sum(path):
    h = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

meta = {
    "version": "v1.0",
    "seed": 2025,
    "input_csv": str(RAW),
    "output_csv": str(PROC),
    "n_rows": int(df.shape[0]),
    "n_cols": int(df.shape[1]),
    "md5_in": md5sum(RAW),
    "md5_out": md5sum(PROC),
    "columns": df.columns.tolist()
}

# Guardar metadata
with open(META, "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("✅ Dataset procesado y metadata creada")
print(f"→ MD5 (input): {meta['md5_in']}")
print(f"→ MD5 (output): {meta['md5_out']}")
print(f"→ Filas: {meta['n_rows']}, Columnas: {meta['n_cols']}")
print("\nDistribución del target 'autorizar':")
print(df['autorizar'].value_counts(dropna=False))
